In [1]:
import os
import keras
import pandas as pd
import numpy as np
import matchzoo as mz

os.environ['CUDA_VISIBLE_DEVICES'] = ''

Using TensorFlow backend.


In [2]:
train_pack = mz.datasets.wiki_qa.load_data('train', task='ranking')
valid_pack = mz.datasets.wiki_qa.load_data('dev', task='ranking', filtered=True)
predict_pack = mz.datasets.wiki_qa.load_data('test', task='ranking', filtered=True)

In [3]:
preprocessor = mz.preprocessors.CDSSMPreprocessor(fixed_length_left=10, fixed_length_right=10)
train_pack_processed = preprocessor.fit_transform(train_pack)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter: 100%|██████████| 2118/2118 [00:00<00:00, 6542.53it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter: 100%|██████████| 18841/18841 [00:05<00:00, 3325.99it/s]
Building Vocabulary from a datapack.: 100%|██████████| 1614998/1614998 [00:00<00:00, 3082231.57it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 7934.07it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 3811.25it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 87315.29it/s]
Processing text_left with chain_transform of NgramLetter => WordHashing: 100%|██████████| 2118/2118 [00:08<00:00, 261.04it/s]
Processing text_right with chain_t

In [4]:
valid_pack_processed = preprocessor.transform(valid_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval: 100%|██████████| 122/122 [00:00<00:00, 7332.17it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval: 100%|██████████| 1115/1115 [00:00<00:00, 4039.28it/s]
Processing text_right with transform: 100%|██████████| 1115/1115 [00:00<00:00, 86562.93it/s]
Processing text_left with chain_transform of NgramLetter => WordHashing: 100%|██████████| 122/122 [00:00<00:00, 249.16it/s]
Processing text_right with chain_transform of NgramLetter => WordHashing: 100%|██████████| 1115/1115 [00:18<00:00, 60.13it/s] 
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval: 100%|██████████| 237/237 [00:00<00:00, 8283.96it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval: 100%|██████████| 2300/2300 [00:01<00:00, 1951.14it/s]
Processing text_right with transform: 100%|███████

In [5]:
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankHingeLoss())
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [6]:
model = mz.models.CDSSM()
model.params['input_shapes'] = preprocessor.context['input_shapes']
model.params['task'] = ranking_task
model.params['filters'] = 64
model.params['kernel_size'] = 3
model.params['strides'] = 1
model.params['padding'] = 'same'
model.params['conv_activation_func'] = 'tanh'
model.params['w_initializer'] = 'glorot_normal'
model.params['b_initializer'] = 'zeros'
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 64
model.params['mlp_num_fan_out'] = 64
model.params['mlp_activation_func'] = 'tanh'
model.params['dropout_rate'] = 0.8
model.params['optimizer'] = 'adadelta'
model.guess_and_fill_missing_params()
model.build()
model.compile()
model.backend.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10, 9644)     0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 10, 9644)     0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 10, 64)       1851712     text_left[0][0]                  
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 10, 64)       1851712     text_right[0][0]                 
__________________________________________________________________________________________________
dropout_1 

In [7]:
pred_x, pred_y = predict_pack_processed[:].unpack()

In [8]:
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=pred_x, y=pred_y, batch_size=len(pred_x))
train_generator = mz.PairDataGenerator(train_pack_processed, num_dup=2, num_neg=1, batch_size=64, shuffle=True)

In [9]:
history = model.fit_generator(train_generator, epochs=20, callbacks=[evaluate], workers=1, use_multiprocessing=False)

Epoch 1/20
32/32 [==============================] - 72s 2s/step - loss: 0.9808
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.42488215100974375 - normalized_discounted_cumulative_gain@5(0.0): 0.4909840291447614 - mean_average_precision(0.0): 0.45215174584733625
Epoch 2/20
32/32 [==============================] - 39s 1s/step - loss: 0.7890
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4523924010888256 - normalized_discounted_cumulative_gain@5(0.0): 0.526580105002082 - mean_average_precision(0.0): 0.4843661871232366
Epoch 3/20
32/32 [==============================] - 39s 1s/step - loss: 0.6269
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.42912083006483787 - normalized_discounted_cumulative_gain@5(0.0): 0.5059856809765926 - mean_average_precision(0.0): 0.46958607676474695
Epoch 4/20
32/32 [==============================] - 40s 1s/step - loss: 0.5425
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.45161347891396936 - normalized_discount